<a href="https://colab.research.google.com/github/waytogoasis/app/blob/main/abc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import auth
import gspread
from google.auth import default
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
from google.colab import drive
import os

# --- 1. Acesso à Planilha e Autenticação ---

def authenticate_and_access_sheet():
    """Autentica o usuário e acessa a planilha 'base'."""
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)

    spreadsheet_name = 'base'
    try:
        spreadsheet = gc.open(spreadsheet_name)
        worksheet = spreadsheet.sheet1
        return worksheet
    except gspread.exceptions.SpreadsheetNotFound:
        print(f"Planilha '{spreadsheet_name}' não encontrada. Verifique o nome e as permissões de compartilhamento.")
        return None

# --- 2. Leitura dos Dados da Planilha ---

def read_data(worksheet):
    """Lê os dados da planilha e os organiza em um dicionário."""
    data = worksheet.get_all_values()
    rows = data[1:]  # Ignora o cabeçalho

    turbines_data = {}
    for row in rows:
        segment_turbine = row[0]
        segment = int(segment_turbine[0])
        turbine = segment_turbine[1]
        if turbine not in turbines_data:
            turbines_data[turbine] = {}
        if segment not in turbines_data[turbine]:
            turbines_data[turbine][segment] = {}

        turbines_data[turbine][segment] = {
            'intensities': [int(val) for val in row[1:]]
        }
    return turbines_data

# --- 3. Implementação do Algoritmo ABC ---

def fitness_function(intensities):
    """Calcula a aptidão de um padrão de excitação (simulação)."""
    # A aptidão é a soma das intensidades, com um bônus para os segmentos mais profundos.
    return sum(intensities) + intensities[3] * 0.5 + intensities[4] * 1

def generate_food_source(num_segments=5):
    """Gera uma fonte de alimento aleatória (solução inicial)."""
    return [random.randint(0, 10) for _ in range(num_segments)]

def generate_neighbor_food_source(food_source, partner_food_source):
    """Gera uma nova fonte de alimento vizinha, baseada na atual e em uma parceira."""
    new_food_source = food_source[:]
    k = random.randint(0, len(food_source) - 1)
    new_food_source[k] = int(max(0, min(10, food_source[k] + random.uniform(-1, 1) * (food_source[k] - partner_food_source[k]))))
    return new_food_source

def calculate_probabilities(food_sources):
    """Calcula as probabilidades de escolha para as abelhas observadoras."""
    fitness_values = [fitness_function(fs) for fs in food_sources]
    total_fitness = sum(fitness_values)
    return [f / total_fitness for f in fitness_values]

def artificial_bee_colony(turbine_data, num_bees=20, max_iterations=100, limit=10):
    """
    Implementa o algoritmo Artificial Bee Colony (ABC) para otimização das intensidades.

    Args:
        turbine_data (dict): Dados da turbina (segmentos e intensidades).
        num_bees (int): Número total de abelhas (empregadas + observadoras).
        max_iterations (int): Número máximo de iterações.
        limit (int): Limite de abandono de uma fonte de alimento.

    Returns:
        dict: Melhor padrão de excitação encontrado (melhor fonte de alimento).
    """

    num_segments = 5
    food_sources = [generate_food_source(num_segments) for _ in range(num_bees // 2)]
    abandonment_counters = [0] * (num_bees // 2)

    best_food_source = food_sources[0]
    best_fitness = fitness_function(best_food_source)

    for iteration in range(max_iterations):
        # Fase das Abelhas Empregadas
        for i in range(num_bees // 2):
            partner = random.choice([p for p in range(num_bees // 2) if p != i])
            new_food_source = generate_neighbor_food_source(food_sources[i], food_sources[partner])
            new_fitness = fitness_function(new_food_source)

            if new_fitness > fitness_function(food_sources[i]):
                food_sources[i] = new_food_source
                abandonment_counters[i] = 0
                if new_fitness > best_fitness:
                    best_fitness = new_fitness
                    best_food_source = new_food_source
            else:
                abandonment_counters[i] += 1

        # Fase das Abelhas Observadoras
        probabilities = calculate_probabilities(food_sources)
        for _ in range(num_bees // 2):
            chosen_index = np.random.choice(range(num_bees // 2), p=probabilities)
            partner = random.choice([p for p in range(num_bees // 2) if p != chosen_index])
            new_food_source = generate_neighbor_food_source(food_sources[chosen_index], food_sources[partner])
            new_fitness = fitness_function(new_food_source)

            if new_fitness > fitness_function(food_sources[chosen_index]):
                food_sources[chosen_index] = new_food_source
                abandonment_counters[chosen_index] = 0
                if new_fitness > best_fitness:
                    best_fitness = new_fitness
                    best_food_source = new_food_source
            else:
                abandonment_counters[chosen_index] += 1

        # Fase das Abelhas Escoteiras
        for i in range(num_bees // 2):
            if abandonment_counters[i] > limit:
                food_sources[i] = generate_food_source(num_segments)
                abandonment_counters[i] = 0

        print(f"Iteração {iteration + 1}: Melhor Aptidão = {best_fitness:.2f}, Melhor Fonte = {best_food_source}")

    return {'best_intensities': best_food_source, 'best_fitness': best_fitness}

# --- 4. Execução do ABC para cada Turbina ---

def run_abc_optimization(turbines_data):
    """Executa o algoritmo ABC para cada turbina e segmento."""
    optimized_turbines = {}
    for turbine, segments in turbines_data.items():
        print(f"Otimizando a turbina {turbine}:")
        optimized_intensities = {}
        for segment, data in segments.items():
            print(f" Otimizando para o segmento {segment}:")
            best_solution = artificial_bee_colony(data, num_bees=20, max_iterations=100, limit=10)
            optimized_intensities[segment] = best_solution['best_intensities']
        optimized_turbines[turbine] = optimized_intensities
    return optimized_turbines

# --- 5. Geração de Gráficos com Estilo e Salvamento na Pasta "abc" ---

def mount_drive_and_configure_paths():
    """Monta o Google Drive e configura os caminhos para salvar os gráficos."""
    drive.mount('/content/drive')
    abc_graficos_path = '/content/drive/MyDrive/1aoPq66KDLN75_5z7vyRbWCWazogtjevV/abc'  # Pasta compartilhada

    if not os.path.exists(abc_graficos_path):
        os.makedirs(abc_graficos_path)

    return abc_graficos_path

def define_graph_style():
    """Define o estilo dos gráficos (fundo preto e cores neon)."""
    plt.style.use('dark_background')
    neon_colors = ['#00FF00', '#00FFFF', '#FF9900', '#FF00FF', '#FFFF00']
    return neon_colors

def generate_visualizations_abc(optimized_turbines, original_data, abc_graficos_path, neon_colors):
    """
    Gera visualizações gráficas dos resultados do ABC e as salva na pasta "abc".

    Args:
        optimized_turbines (dict): Resultados otimizados do ABC.
        original_data (list): Dados originais da planilha.
        abc_graficos_path (str): Caminho para a pasta onde os gráficos serão salvos.
        neon_colors (list): Lista de cores neon.
    """

    original_intensities = {}
    for row in original_data[1:]:
        segment_turbine, *intensities = row
        turbine = segment_turbine[1]
        segment = int(segment_turbine[0])
        if turbine not in original_intensities:
            original_intensities[turbine] = {}
        original_intensities[turbine][segment] = [int(val) for val in intensities]

    # Gráfico 1
    plt.figure(figsize=(12, 6))
    for i, turbine in enumerate(sorted(optimized_turbines.keys())):
        original_avg = [np.mean(original_intensities[turbine][seg]) for seg in sorted(original_intensities[turbine].keys())]
        optimized_avg = [np.mean(optimized_turbines[turbine][seg]) for seg in sorted(optimized_turbines[turbine].keys())]
        plt.plot(original_avg, label=f'Original - Turbina {turbine}', color=neon_colors[i%5], linestyle='--')
        plt.plot(optimized_avg, label=f'ABC - Turbina {turbine}', color=neon_colors[i%5])
    plt.xlabel('Segmento')
    plt.ylabel('Intensidade Média')
    plt.title('Comparação das Intensidades Médias por Turbina')
    plt.legend()
    plt.savefig(f'{abc_graficos_path}/grafico_1.png')
    plt.close()
    display(Image(filename=f'{abc_graficos_path}/grafico_1.png'))

    # Gráfico 2
    plt.figure(figsize=(10, 5))
    segments = sorted(optimized_turbines['a'].keys())
    intensities = [optimized_turbines['a'][seg] for seg in segments]
    plt.bar(segments, [np.mean(i) for i in intensities], color=neon_colors[0])
    plt.xlabel('Segmento')
    plt.ylabel('Intensidade Média')
    plt.title('Intensidades Otimizadas por Segmento (Turbina a) - ABC')
    plt.savefig(f'{abc_graficos_path}/grafico_2.png')
    plt.close()
    display(Image(filename=f'{abc_graficos_path}/grafico_2.png'))

    # Gráfico 3
    plt.figure(figsize=(10, 5))
    original_b = [np.mean(original_intensities['b'][seg]) for seg in sorted(original_intensities['b'].keys())]
    optimized_b = [np.mean(optimized_turbines['b'][seg]) for seg in sorted(optimized_turbines['b'].keys())]
    plt.plot(original_b, label='Original', color=neon_colors[1], linestyle='--')
    plt.plot(optimized_b, label='ABC', color=neon_colors[1])
    plt.xlabel('Segmento')
    plt.ylabel('Intensidade Média')
    plt.title('Comparação das Intensidades por Segmento (Turbina b)')
    plt.legend()
    plt.savefig(f'{abc_graficos_path}/grafico_3.png')
    plt.close()
    display(Image(filename=f'{abc_graficos_path}/grafico_3.png'))

    # Gráfico 4
    plt.figure(figsize=(8, 6))
    all_intensities = [[np.mean(optimized_turbines[turbine][seg]) for seg in sorted(optimized_turbines[turbine].keys())] for turbine in sorted(optimized_turbines.keys())]
    plt.imshow(all_intensities, cmap='magma', aspect='auto')
    plt.colorbar(label='Intensidade Média')
    plt.yticks(np.arange(len(optimized_turbines.keys())), labels=sorted(optimized_turbines.keys()))
    plt.xticks(np.arange(5), labels=['1', '2', '3', '4', '5'])
    plt.xlabel('Segmento')
    plt.ylabel('Turbina')
    plt.title('Mapa de Calor das Intensidades Otimizadas - ABC')
    plt.savefig(f'{abc_graficos_path}/grafico_4.png')
    plt.close()
    display(Image(filename=f'{abc_graficos_path}/grafico_4.png'))

    # Gráfico 5
    fig, axs = plt.subplots(2, 3, figsize=(18, 10), sharey=True)
    fig.suptitle('Dispersão entre Intensidade Média e Segmento para cada Turbina')
    for i, turbine in enumerate(sorted(optimized_turbines.keys())):
        row, col = i // 3, i % 3
        intensities = [np.mean(optimized_turbines[turbine][seg]) for seg in sorted(optimized_turbines[turbine].keys())]
        axs[row, col].scatter(range(1, 6), intensities, color=neon_colors[i%5])
        axs[row, col].set_title(f'Turbina {turbine}')
        axs[row, col].set_xlabel('Segmento')
        axs[row, col].set_ylabel('Intensidade Média')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(f'{abc_graficos_path}/grafico_5.png')
    plt.close()
    display(Image(filename=f'{abc_graficos_path}/grafico_5.png'))

    # Gráfico 6
    plt.figure(figsize=(8, 5))
    intensities_e = [item for sublist in [optimized_turbines['e'][seg] for seg in sorted(optimized_turbines['e'].keys())] for item in sublist]
    plt.hist(intensities_e, bins=10, color=neon_colors[4], edgecolor='black')
    plt.xlabel('Intensidade')
    plt.ylabel('Frequência')
    plt.title('Histograma das Intensidades para a Turbina e - ABC')
    plt.savefig(f'{abc_graficos_path}/grafico_6.png')
    plt.close()
    display(Image(filename=f'{abc_graficos_path}/grafico_6.png'))

    # Gráfico 7
    plt.figure(figsize=(10, 6))
    plt.boxplot(list(map(list, zip(*all_intensities))), labels=['1', '2', '3', '4', '5'], patch_artist=True,
                boxprops=dict(facecolor=neon_colors[2], color='white'),
                capprops=dict(color='white'),
                whiskerprops=dict(color='white'),
                flierprops=dict(color='white', markeredgecolor='white'),
                medianprops=dict(color='white'))
    plt.xlabel('Segmento')
    plt.ylabel('Intensidade')
    plt.title('Boxplot das Intensidades por Segmento - ABC')
    plt.savefig(f'{abc_graficos_path}/grafico_7.png')
    plt.close()
    display(Image(filename=f'{abc_graficos_path}/grafico_7.png'))